# Learning from Imbalanced Data

과제내용
- Imbalanced learning 기법 간 성능 비교
- 상세내용은 아래 실험방법 참조


•데이터
- Kaggle Competition으로 2017년말에 개최되었던 "WSDM - KKBox's Music Recommendation Challenge" 데이터에 Feature Engineering을 적용한 것을 사용한다. 
(첨부화일)


•실험방법
- class 변수인 'target' 필드에서 1의 값이 전체 데이터의 0.1%, 1%, 10%가 되도록 축소하여 3가지의 data imbalance 상황을 만든다.
(필요시 위에 더하여 새로운 imbalance 상황을 추가할 수 있음)
- 각각의 data imbalance 상황에서, 여러 Imbalanced learning 기법을 적용한 데이터에 대해  RandomForest 알고리즘을 적용하여 성능을 평가하고 비교한다. 
(존재하는 feature를 수정하지 않고 그대로 사용함)
- 평가척도로는 Accuracy, ROC-AUC, Recall, Precision, F1-score 등 5가지를 사용하고, "Survey of resampling techniques for improving classification performance in unbalanced datasets" 논문(첨부화일)에서 테스트한 기법 중 imbalanced-learn 패키지에서 지원하는 것은 모두 테스트한다.


•제출물
- 소스코드(화일명: 이름1-이름2.ipynb)와 결과보고서(화일명: 이름1-이름2.ppt)
- 4월22일 수업시간에 팀당 10분 이내 발표


•유의사항
- 마감시간 내에 제출한 과제만 평가함. 
- 반드시, 담당교수 이메일(www4u@kookmin.ac.kr)로 제출해야 하고, 메일 제목은 "팀과제(2차) 이름1-이름2"로 할 것.


In [3]:
# ! pip install -U imbalanced-learn # U 하면 새버전으로 깔림.

In [4]:
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from collections import Counter
from sklearn.svm import SVC

In [5]:
data = pd.read_csv("WSDM_train_full_features.csv")

In [6]:
data.head()

,target,user_idx,song_idx,ssn_idx,sst_idx,st_idx,song_length,genr_idx,art_idx,com_idx,...,lyricists_count,composer_count,is_featured,artist_count,artist_composer,artist_composer_lyricist,song_lang_boolean,smaller_song,count_song_played,count_artist_played
0,1,0,0,0,0,0,206471,7,1877,7450,...,0,2,0,0,0,0,0,1,215,1140
1,1,1,1,1,1,1,284584,18,92,2,...,0,1,0,0,0,0,0,0,1,303616
2,1,1,2,1,1,1,225396,18,527,100234,...,0,1,0,0,0,0,0,1,4,289
3,1,1,3,1,1,1,255512,229,89592,193180,...,0,1,0,0,0,0,0,0,1,1
4,1,0,4,0,0,0,187802,13,195,137,...,0,3,0,0,0,0,0,1,412,427


In [7]:
round(len(data)/2)

3688709

In [8]:
data=data.iloc[0:round(len(data)/2),:]
data.shape

(3688709, 30)

In [9]:
data=data.iloc[0:round(len(data)/2),:]
data.shape

(1844354, 30)

In [10]:
data_0 = data[data['target']==0]
data_1 = data[data['target']==1]

In [11]:
print(data_0.shape[0])
print(data_1.shape[0])  # 비슷한 크기 -> 둘 중 하나의 값을 줄여야한다.

643094
1201260


In [12]:
data_1_01 = data_1.iloc[np.random.randint(0,len(data_1),size=round((1/9)*len(data_0)))]
data_1_01

,target,user_idx,song_idx,ssn_idx,sst_idx,st_idx,song_length,genr_idx,art_idx,com_idx,...,lyricists_count,composer_count,is_featured,artist_count,artist_composer,artist_composer_lyricist,song_lang_boolean,smaller_song,count_song_played,count_artist_played
1570756,1,4112,5885,1,1,2,241998,0,7,181,...,0,1,0,0,0,0,0,1,939,115325
811027,1,8892,14262,1,1,2,212323,10,2095,2,...,0,1,0,0,0,0,0,1,109,2427
1613850,1,4635,51109,0,0,4,243043,0,3118,2,...,0,1,0,0,0,0,0,1,20,738
1330550,1,8073,246,1,1,1,202368,4,28,26,...,0,1,0,0,0,0,0,1,8062,182088
1620579,1,18417,972,1,1,2,212750,97,1146,801,...,3,4,0,0,0,0,0,1,10565,36781
91002,1,4244,10590,1,1,2,309185,0,265,1328,...,0,1,0,0,0,0,0,0,358,49055
1692543,1,11110,6332,1,1,2,213733,0,1441,2,...,0,1,0,0,0,0,0,1,128,1923
325981,1,11455,819,1,1,1,186850,0,69,2,...,0,1,0,0,0,0,0,1,1053,8843
838689,1,849,34033,3,1,2,228101,0,406,2,...,0,1,0,0,0,0,0,1,189,3860
990986,1,10467,16054,12,6,8,239177,4,277,187,...,0,1,0,0,0,0,0,1,262,3707


In [13]:
df = pd.concat([data_0,data_1_01],axis=0)
df.head()
df.shape

(714549, 30)

In [14]:
data.shape # 원본데이터

(1844354, 30)

In [15]:
X = df.iloc[:,1:30]
y = df.iloc[:,0]

In [16]:
# 학습/평가 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape

((535911, 29), (178638, 29))

# Under_Sampling
# 2-1. NearMiss1. 

In [17]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1,random_state=0)

In [18]:
X_resampled_nm1, y_resampled = nm1.fit_sample(X_train, y_train)
X_resampled_nm1.shape

(107328, 29)

In [19]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled_nm1, y_resampled).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(0.26461335214232135,
 0.47473256942771025,
 0.1664255437955278,
 0.7371142712607498,
 0.09380208147061979)

# 2-2. NearMiss2. 

In [20]:
from imblearn.under_sampling import NearMiss
nm2 = NearMiss(version=2,random_state=0)

In [21]:
X_resampled_nm2, y_resampled = nm2.fit_sample(X_train, y_train)
X_resampled_nm2.shape

MemoryError: 

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled_nm2, y_resampled).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 2-3. NearMiss3. 

In [22]:
from imblearn.under_sampling import NearMiss
nm3 = NearMiss(version=3,random_state=0)

In [23]:
X_resampled_nm3, y_resampled = nm3.fit_sample(X_train, y_train)
X_resampled_nm3.shape

(107328, 29)

In [24]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled_nm3, y_resampled).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(0.5990662680952541,
 0.6264866907657122,
 0.24713030315771767,
 0.6607273340453038,
 0.151989242445792)

# 3. RandomUnderSampler

In [25]:
from imblearn.under_sampling import RandomUnderSampler 

In [26]:
rus = RandomUnderSampler(random_state=0)

In [ ]:
X_res, y_res = rus.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(0.6976455177509824,
 0.6597569727502797,
 0.28747823333861006,
 0.6124444944072845,
 0.1878199713857239)

# 4. Condensed Nearest Neighbor (CNN)

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour

In [ ]:
cnn = CondensedNearestNeighbour(random_state=0)

In [ ]:
X_res, y_res = cnn.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 5. Edited Nearest Neighbor (ENN)

In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours

In [ ]:
enn = EditedNearestNeighbours(random_state=0)

In [ ]:
X_res, y_res = enn.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 6. Repeated Edited Nearest Neighbor 

In [ ]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours 

In [ ]:
renn = RepeatedEditedNearestNeighbours(random_state=0)

In [ ]:
X_res, y_res = renn.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 7. Tomek Link Removal

In [ ]:
from imblearn.under_sampling import TomekLinks 

In [ ]:
tl = TomekLinks(random_state=0)

In [ ]:
X_res, y_res = tl.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# Oversampling methods
# 1. Random oversampling of minority class

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state=0)

In [ ]:
X_res, y_res = ros.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 2. SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled, y_resampled).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 3-1. Borderline-SMOTE1

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE(kind='Borderline-1',random_state=0).fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled, y_resampled).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 3-2.Borderline-SMOTE2

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE(kind='Borderline-2',random_state=0).fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled, y_resampled).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# Combination methods
# 1. SMOTE + Tomek Link Removal

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
smt = SMOTETomek(random_state=0)

In [ ]:
 X_res, y_res = smt.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 2. SMOTE + ENN

In [ ]:
 from imblearn.combine import SMOTEENN

In [ ]:
sme = SMOTEENN(random_state=42)

In [ ]:
X_res, y_res = sme.fit_resample(X_train, y_train)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# Ensemble Methods
# 1. EasyEnsemble

In [ ]:
from imblearn.ensemble import EasyEnsemble

In [ ]:
ee = EasyEnsemble(random_state=0t)

In [ ]:
X_res, y_res = ee.fit_resample(X, y)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)

# 2. BalanceCascade

In [ ]:
from imblearn.ensemble import BalanceCascade

In [ ]:
bc = BalanceCascade(random_state=42)

In [ ]:
X_res, y_res = bc.fit_resample(X, y)

In [ ]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)